In [ ]:
import anndata as ad
import numpy as np
import pandas as pd
import scanpy as sc
import os


In [4]:
adata = sc.read_h5ad("adata_objects/h5ad/WM_Post_Expression_Correction/All_Samples.h5ad")

In [ ]:
adata = adata[
    (adata.obs["Treatment"] == "Control") &
    (adata.obs["Subregion"].notna()) &
    (adata.obs["Subregion"] != "Other")
]
adata.var_names_make_unique()
adata.var["mt"] = adata.var_names.str.startswith("mt-")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], inplace=True)
sc.pp.filter_genes(adata, min_counts = 25)
sc.pp.normalize_total(adata, inplace=True)
sc.pp.log1p(adata)

sc.tl.rank_genes_groups( 
    adata,  
    groupby="Subregion", 
    method='wilcoxon'
) 

In [ ]:

for subregion in adata.obs["Subregion"].unique():

    dge_df = sc.get.rank_genes_groups_df(adata, group= subregion)

    path = f"DGE_Data/Control_Only_Subclusters/"
    if not os.path.exists(path):
        os.makedirs(path)
    if subregion == "Midbrain/Hindbrain":
        dge_df.to_csv(f"DGE_Data/Control_Only_Subclusters/Midbrain_Hindbrain.csv")     
    else:
        dge_df.to_csv(f"DGE_Data/Control_Only_Subclusters/{subregion}.csv")